# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 11 2022 12:59PM,250610,19,TR-RM-11142022,"GCH Granules USA, Inc.",Released
1,Nov 11 2022 12:45PM,250608,10,0086163940,ISDIN Corporation,Released
2,Nov 11 2022 12:45PM,250608,10,0086163946,ISDIN Corporation,Released
3,Nov 11 2022 12:45PM,250608,10,0086163947,ISDIN Corporation,Released
4,Nov 11 2022 12:45PM,250608,10,0086163949,ISDIN Corporation,Released
5,Nov 11 2022 12:45PM,250608,10,0086163950,ISDIN Corporation,Released
6,Nov 11 2022 12:45PM,250608,10,0086163951,ISDIN Corporation,Released
7,Nov 11 2022 12:45PM,250608,10,0086163941,ISDIN Corporation,Released
8,Nov 11 2022 12:45PM,250608,10,0086163939,ISDIN Corporation,Released
9,Nov 11 2022 12:45PM,250608,10,0086163943,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,250604,Released,1
35,250606,Released,4
36,250607,Released,6
37,250608,Released,13
38,250610,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250604,NaN,NaN,1.0
250606,NaN,NaN,4.0
250607,NaN,NaN,6.0
250608,NaN,NaN,13.0
250610,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250428,0.0,1.0,0.0
250518,0.0,0.0,5.0
250534,2.0,0.0,10.0
250545,0.0,0.0,1.0
250546,7.0,5.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250428,0,1,0
250518,0,0,5
250534,2,0,10
250545,0,0,1
250546,7,5,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250428,0,1,0
1,250518,0,0,5
2,250534,2,0,10
3,250545,0,0,1
4,250546,7,5,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250428,,1,
1,250518,,,5
2,250534,2,,10
3,250545,,,1
4,250546,7,5,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc."
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation
14,Nov 11 2022 12:40PM,250607,12,Hush Hush
20,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation
24,Nov 11 2022 12:35PM,250604,15,Lonza
25,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc."
26,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC"
27,Nov 11 2022 12:10PM,250601,22,FUJIFILM Diosynth Biotechnologies Texas LLC
28,Nov 11 2022 12:06PM,250600,10,ISDIN Corporation
50,Nov 11 2022 12:01PM,250598,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",,,1
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,,,13
2,Nov 11 2022 12:40PM,250607,12,Hush Hush,,,6
3,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,,,4
4,Nov 11 2022 12:35PM,250604,15,Lonza,,,1
5,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",,,1
6,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",,,1
7,Nov 11 2022 12:10PM,250601,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
8,Nov 11 2022 12:06PM,250600,10,ISDIN Corporation,,,22
9,Nov 11 2022 12:01PM,250598,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1,,
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,13,,
2,Nov 11 2022 12:40PM,250607,12,Hush Hush,6,,
3,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,4,,
4,Nov 11 2022 12:35PM,250604,15,Lonza,1,,
5,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",1,,
6,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",1,,
7,Nov 11 2022 12:10PM,250601,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
8,Nov 11 2022 12:06PM,250600,10,ISDIN Corporation,22,,
9,Nov 11 2022 12:01PM,250598,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1,,
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,13,,
2,Nov 11 2022 12:40PM,250607,12,Hush Hush,6,,
3,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,4,,
4,Nov 11 2022 12:35PM,250604,15,Lonza,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,13.0,NaN,NaN
2,Nov 11 2022 12:40PM,250607,12,Hush Hush,6.0,NaN,NaN
3,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,4.0,NaN,NaN
4,Nov 11 2022 12:35PM,250604,15,Lonza,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,13.0,0.0,0.0
2,Nov 11 2022 12:40PM,250607,12,Hush Hush,6.0,0.0,0.0
3,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,4.0,0.0,0.0
4,Nov 11 2022 12:35PM,250604,15,Lonza,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4510383,84.0,9.0,15.0
12,250607,6.0,0.0,0.0
15,751763,10.0,18.0,0.0
16,501019,1.0,2.0,0.0
19,1002385,4.0,0.0,0.0
21,751702,3.0,0.0,0.0
22,250601,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4510383,84.0,9.0,15.0
1,12,250607,6.0,0.0,0.0
2,15,751763,10.0,18.0,0.0
3,16,501019,1.0,2.0,0.0
4,19,1002385,4.0,0.0,0.0
5,21,751702,3.0,0.0,0.0
6,22,250601,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,84.0,9.0,15.0
1,12,6.0,0.0,0.0
2,15,10.0,18.0,0.0
3,16,1.0,2.0,0.0
4,19,4.0,0.0,0.0
5,21,3.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,6.0
2,15,Released,10.0
3,16,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,15.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,18.0,2.0,0.0,0.0,0.0
Released,84.0,6.0,10.0,1.0,4.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,15.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,18.0,2.0,0.0,0.0,0.0
2,Released,84.0,6.0,10.0,1.0,4.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,15.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,18.0,2.0,0.0,0.0,0.0
2,Released,84.0,6.0,10.0,1.0,4.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()